In [6]:
import re
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

import datetime

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [28]:
def free_rooms2(corp=''):
    # corp = Корпус {}, where {} is it's name
    
    now = datetime.datetime.now()
    time_str = str(now.hour)+str(':')+str(now.minute)

    links = []
    free_rooms = []

    r = requests.get("https://home.mephi.ru/rooms")
    b = BeautifulSoup(r.text, "html.parser")
    
    links_block = b.select('#page-content-wrapper > div > div')[0].find(string=corp).findNext("ul").find_all("li")

    for link in links_block:
        links.append(["https://home.mephi.ru" + link.a.attrs['href'] + "/day", link.get_text()])
        
          
    def parse_link(link):
        r = requests.get(link[0])
        b = BeautifulSoup(r.text, "html.parser")
        try:
            shedule_block = b.select('#page-content-wrapper > div > div:nth-child(6) > div')[0].get_text()
            shedule_time = [s.replace("\xa0— ", "_").replace("\xa0", "_") for s in shedule_block.split('\n') if (len(s)>=3) and (s[2]==":")]
            
            if (len(list(filter(lambda x: (x[:5] <= time_str) and (time_str <= x[-5:]), shedule_time))) == 0) and ("каф" not in link[1]):
                return link[1]
            
        except IndexError:
            if (b.select('#page-content-wrapper > div > div.alert.alert-info')[0].get_text() == 'Занятий не найдено') and ("каф" not in link[1]):
                return link[1]
            
        except BaseException:
            pass
            
            
    pool = ThreadPoolExecutor(5)
    futures = []
            
    for link in links:
        futures.append(pool.submit(parse_link, link))
        
    for x in as_completed(futures):
        free_rooms.append(x.result())

    #print(free_rooms)
    return [x for x in free_rooms if x is not None]

In [32]:
# from concurrent.futures import ThreadPoolExecutor, wait, as_completed
 
# def return_after_5_secs(num):
#     sleep(randint(1, 5))
#     #return "Return of {}".format(num)
#     return num

# pool = ThreadPoolExecutor(5)

# futures = []

# for x in range(5):
#     futures.append(pool.submit(return_after_5_secs, x))

# # r = []
# # for x in as_completed(futures):
# #     r.append(x.result())

# # print(r)


In [31]:
%%time
fr = free_rooms2('Корпус Б')
print(len(fr))
print(fr)

37
['Б-105', 'Б-039', 'Б-108', 'Б-106А', 'Б-100', 'Б-118', 'Б-109', 'Б-124/126', 'Б-201', 'Б-202', 'Б-205', 'Б-207', 'Б-211', 'Б-208', 'Б-210', 'Б-212', 'Б-212ст', 'Б-213', 'Б-214', 'Б-215', 'Б-216', 'Б-217', 'Б-218', 'Б-221', 'Б-219', 'Б-303', 'Б-301', 'Б-304', 'Б-308', 'Б-314а', 'Б-314б', 'Б-316', 'Б-315', 'Б-317', 'Б-319', 'Б-401', 'Б-403']
CPU times: user 1.16 s, sys: 37.3 ms, total: 1.19 s
Wall time: 3.54 s


In [33]:
# import concurrent.futures
# URLS = [1, 2, 3, 4, 5]

# # Retrieve a single page and report the url and contents
# def load_url(n):
#     sleep(randint(1, 5))
#     return n**2
    
# r = []
# # We can use a with statement to ensure threads are cleaned up promptly
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     # Start the load operations and mark each future with its URL
#     future_to_url = [executor.submit(load_url, url) for url in URLS]
#     for future in concurrent.futures.as_completed(future_to_url):
#         r.append(future.result())
        
# print(r)

In [24]:
def free_rooms(corp=''):
    # corp = Корпус {}, where {} is it's name
    
    now = datetime.datetime.now()
    time_str = str(now.hour)+str(':')+str(now.minute)

    links = []
    free_rooms = []

    r = requests.get("https://home.mephi.ru/rooms")
    b = BeautifulSoup(r.text, "html.parser")
    
    links_block = b.select('#page-content-wrapper > div > div')[0].find(string=corp).findNext("ul").find_all("li")

    for link in links_block:
        links.append(["https://home.mephi.ru" + link.a.attrs['href'] + "/day", link.get_text()])

    for link in links:
        r = requests.get(link[0])
        b = BeautifulSoup(r.text, "html.parser")
        try:
            shedule_block = b.select('#page-content-wrapper > div > div:nth-child(6) > div')[0].get_text()
            shedule_time = [s.replace("\xa0— ", "_").replace("\xa0", "_") for s in shedule_block.split('\n') if (len(s)>=3) and (s[2]==":")]
        except IndexError:
            if (b.select('#page-content-wrapper > div > div.alert.alert-info')[0].get_text() == 'Занятий не найдено') and ("каф" not in link[1]):
                free_rooms.append(link[1])
                continue
        except BaseException: 
            continue

        if (len(list(filter(lambda x: (x[:5] <= time_str) and (time_str <= x[-5:]), shedule_time))) == 0) and ("каф" not in link[1]):
            free_rooms.append(link[1])

    #print(free_rooms)
    return free_rooms

# обработать ошибку none type object has no atribute findNext

# обработать ошибку с последним кабинетом - не фильтрует каф

In [25]:
# _date = datetime.datetime.now().strftime("%Y-%m-%d")
    
# url = "https://home.mephi.ru/study_groups/5256/day?date=2019-02-11"

# r = requests.get(url)
# b = BeautifulSoup(r.text, "html.parser")

# try:
#     shedule_block = b.select('#page-content-wrapper > div > div:nth-child(7) > div')[0].get_text()
#     shedule_txt = [s.replace("\xa0— ", "_").replace("\xa0", "_") for s in shedule_block.split('\n') if s != '']

# # на случай воскресенья или праздника
# except BaseException:
#     print('empty')

# def parse_test_shadule(test):
#     numbers = [str(i) for i in range(10)]
#     hours_d = {}

#     for i, el in enumerate(test):
#         if (len(el) > 3) and (el[0] in numbers) and (el[1] in numbers):
#             lesson_time = el
#             hours_d[lesson_time] = []
#             continue
#         hours_d[lesson_time].append(el)

#     def split_in_two(arr):
#         ix = len(arr) // 2
#         return [arr[:ix], arr[ix:]]

#     for key in hours_d.keys():
#         if len(hours_d[key]) > 5:
#             hours_d[key] = split_in_two(hours_d[key])

#     return(hours_d)

# d = parse_test_shadule(shedule_txt)

# parse_lesson(d)

In [55]:
def parse_shedule_today(datatime_date=None):
    #_date = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    _date = datetime.datetime.now().date() if not datatime_date else datatime_date
    _date = _date.strftime("%Y-%m-%d")
    
    url = "https://home.mephi.ru/study_groups/5256/day?date=" + _date
    
    r = requests.get(url)
    b = BeautifulSoup(r.text, "html.parser")
    
    try:
        shedule_block = b.select('#page-content-wrapper > div > div:nth-child(7) > div')[0].get_text()
        shedule_txt = [s.replace("\xa0— ", "_").replace("\xa0", "_") for s in shedule_block.split('\n') if s != '']
        
    # на случай воскресенья или праздника
    except BaseException:
        return {}

    def parse_test_shadule(test):
        numbers = [str(i) for i in range(10)]
        hours_d = {}

        for i, el in enumerate(test):
            if (len(el) > 3) and (el[0] in numbers) and (el[1] in numbers):
                lesson_time = el
                hours_d[lesson_time] = []
                continue
            hours_d[lesson_time].append(el)

        def split_in_two(arr):
            ix = len(arr) // 2
            return [arr[:ix], arr[ix:]]

        for key in hours_d.keys():
            if len(hours_d[key]) > 5:
                hours_d[key] = split_in_two(hours_d[key])

        return(hours_d)


    return parse_test_shadule(shedule_txt)


def parse_lesson(shadule_today):
    now = datetime.datetime.now()
    time_str = str(now.hour)+str(':')+str(now.minute)

    for k in shadule_today.keys():
        if (time_str > k[:5]) and (time_str < k[-5:]):
            return shadule_today[k]

In [59]:
parse_lesson(parse_shedule_today())

['Т-215', 'Лек', 'Безопасность сетей ЭВМ', 'Мельников_Д.А.']

In [8]:


def parse_prof_shedule(name="Мельников Дмитрий Анатольевич", FULL_NAME=True, date=None):
    sec_name = name.split()[0] if FULL_NAME else name   #обработать
    date = str(datetime.datetime.now().hour)+str(':')+str(datetime.datetime.now().minute) if not date else date
    base_url = "https://home.mephi.ru"
    url = "https://home.mephi.ru/tutors?utf8=✓&locale=&term_id=7&query=" + sec_name
    
    try:
        
        b = BeautifulSoup(requests.get(url).text, "html.parser")
        links_corp = b.select('#page-content-wrapper > div > div > div.list-group')[0].find_all('a')
        #prof_links = [base_url + link['href'].split("?")[0]]

        _url = ""
        for link in links_corp:
            if sec_name.lower() in link.text.lower():
                _url = base_url + link['href'].split("?")[0] + "/day?date=" + date

        b = BeautifulSoup(requests.get(_url).text, "html.parser")

        shedule_block = b.select("#page-content-wrapper > div > div:nth-child(6) > div")[0].get_text()
        shedule_txt = [s.replace("\xa0— ", "_").replace("\xa0", "_") for s in shedule_block.split('\n') if (s != "") and (s != ",")]

        def parse_test_shadule(test):
            numbers = [str(i) for i in range(10)]
            hours_d = {}

            for i, el in enumerate(test):
                if (len(el) > 3) and (el[0] in numbers) and (el[1] in numbers):
                    lesson_time = el
                    hours_d[lesson_time] = []
                    continue
                hours_d[lesson_time].append(el)

            return(hours_d)

        return(parse_test_shadule(shedule_txt))

    except BaseException as e:
        print(e)
        return {}
    
    
parse_prof_shedule(name="Велигура", date="2019-02-21")

#доделать разветвление если видит нескольких преподавателей

{'10:15_11:50': ['Б-215',
  'Лек',
  'Системы аналитических вычислений',
  'Б16-505',
  'Б16-565']}